# Homework 01-5 - 下載 Gossiping 板塊文章的留言

## Import the packages

載入 Pandas、Requests 和 BeautifulSoup。

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Tag

## Extract articles

抽象一個 `extract_articles` 函式，擷取文章列表中的文章概要。

- 文章標題：使用 `div.title` 選取
- 文章作者：使用 `div.author` 選取
- 文章日期：使用 `div.date` 選取
- 連結：使用 `div.title` 的 `a` 子元素選取。因為連結是純 pathname，所以要補上 `https://www.ptt.cc` 網域。

In [2]:
def extract_articles(soup: BeautifulSoup) -> pd.DataFrame:
    articles = soup.find_all("div", class_="r-ent")
    results = []

    for i, article in enumerate(articles):
        assert isinstance(article, Tag)

        title = article.find("div", class_="title")
        author = article.find("div", class_="author")
        date = article.find("div", class_="date")
        link = article.select_one(".title > a")

        assert isinstance(title, Tag)
        assert isinstance(author, Tag)
        assert isinstance(date, Tag)

        if link is None:
            print(f"Article {i} has no link")
            continue

        assert isinstance(link, Tag)

        title_text = title.text.strip()
        author_text = author.text.strip()
        date_text = date.text.strip()
        link_text = link.get("href")

        assert isinstance(link_text, str)

        results.append({
            "文章標題": title_text,
            "作者": author_text,
            "文章日期": date_text,
            "文章連結": "https://www.ptt.cc" + link_text
        })

    results_df = pd.DataFrame(results)

    return results_df 

## Extract comments from an article

給定一個內文，取得內文的留言資訊。

- 連結：作為 key 來和 `extract_articles` 的 dataframe 做合併
- 喜好程度：推 / 踩 / →
- 留言者
- 留言內容：移除前面的「: 」。
- 留言時間（不含 IP）
    - 使用 Regex 分離 IP 和日期時間 `/([\d.]+) (\d{2}/\d{2} \d{2}:\d{2})`
    - 只留日期時間

In [3]:
import re

# 111.240.96.24 03/29 22:49
IP_DATETIME_EXTRACTOR = re.compile(r"([\d.]+) (\d{2}/\d{2} \d{2}:\d{2})")
COMMENT_CONTENT_EXTRACTOR = re.compile(r": (.*)")

def extract_comments(link: str, soup: BeautifulSoup) -> pd.DataFrame:
    comments = soup.find_all("div", class_="push")
    results = []

    for comment in comments:
        assert isinstance(comment, Tag)

        # 推 / 踩 / →
        push_tag = comment.find("span", class_="push-tag")
        assert isinstance(push_tag, Tag)
        push_tag_text = push_tag.text.strip()
        assert isinstance(push_tag_text, str)

        # 留言者
        push_user_id = comment.find("span", class_="push-userid")
        assert isinstance(push_user_id, Tag)
        push_user_id_text = push_user_id.text.strip()
        assert isinstance(push_user_id_text, str)

        # 留言內容
        push_content = comment.find("span", class_="push-content")
        assert isinstance(push_content, Tag)
        push_content_text_raw = push_content.text.strip()
        assert isinstance(push_content_text_raw, str)

        push_content_matches = COMMENT_CONTENT_EXTRACTOR.search(push_content_text_raw)
        if push_content_matches is None:
            print(f"warning: push_content_text does not start with ':', ignoring: {push_content_text_raw}")
            continue
        assert isinstance(push_content_matches, re.Match)
        push_content_text = push_content_matches.group(1)
        assert isinstance(push_content_text, str)

        # 留言時間（含 IP）
        push_ipdatetime = comment.find("span", class_="push-ipdatetime")
        assert isinstance(push_ipdatetime, Tag)
        push_ipdatetime_text = push_ipdatetime.text.strip()
        assert isinstance(push_ipdatetime_text, str)

        push_datetime_matches = IP_DATETIME_EXTRACTOR.search(push_ipdatetime_text)
        assert isinstance(push_datetime_matches, re.Match)
        push_datetime = push_datetime_matches.group(2)
        assert isinstance(push_datetime, str)
        
        results.append({
            "連結": link,
            "喜好程度": push_tag_text,
            "留言者": push_user_id_text,
            "留言內容": push_content_text,
            "留言時間": push_datetime
        })

    return pd.DataFrame(results)

## Get previous page URL

抽象一個 `get_previous_page_url` 方法，從網頁中擷取「上一頁」的連結。

In [4]:
def get_previous_page_url(soup: BeautifulSoup) -> str:
    # 抓取 innerText 是 '‹ 上頁' 的元素
    prev_page_link = soup.find("a", string="‹ 上頁")
    assert isinstance(prev_page_link, Tag)

    prev_page_path = prev_page_link.get('href')
    assert isinstance(prev_page_path, str)

    return "https://www.ptt.cc" + prev_page_path

## Main application

取得 Gossiping 的最新一頁內容。

- 需要傳入 cookie `over18=1`，以免卡在驗證是否 18 歲的介面。
- 使用 `lxml` 取得比較快的解析速度。

In [5]:
# First Page

url = "https://www.ptt.cc/bbs/Gossiping/index.html"
resp = requests.get(url, headers={"cookie": "over18=1"})
soup = BeautifulSoup(resp.text, "lxml")

接著使用剛才的函式，取得最新一頁的文章。

In [6]:
pd_1 = extract_articles(soup)
pd_1

,文章標題,作者,文章日期,文章連結
0,[新聞] 登台大港開唱喊「音樂是政治」 楊舒雅,VB6,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415548....
1,Re: [新聞] 曹興誠：若國號改「台灣共和國」是被中共,kaky,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415552....
2,[問卦] 笑哈哈LOL江老師跟Eric為什麼分道揚鑣？,shokotan,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415662....
3,[問卦] 捷運突然停車該注意什麼？,chen0625,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415770....
4,[問卦] 北捷藍線有卦嗎,MeeToo,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415894....
5,Re: [新聞] 網友問郭正亮「兩岸談判談什麼？」 黃,winnabe,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415934....
6,[新聞] 紀念日及節日修法初審通過 關鍵條文送協,ytkuang,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415992....
7,[問卦] 黑人能吃辣嗎？,kerorobear,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743416007....
8,[問卦] 吃洋芋片會胖 不覺得完全沒道理嗎？,wang111283,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743416031....
9,[問卦] 柏原崇當年大概有多紅？,yokann,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743416146....


接著取得上一頁的 URL。

In [7]:
previous_page_url = get_previous_page_url(soup)
previous_page_url

'https://www.ptt.cc/bbs/Gossiping/index38969.html'

取得 Gossiping 的上一頁內容。

In [8]:
# Second Page
resp = requests.get(previous_page_url, headers={"cookie": "over18=1"})
soup = BeautifulSoup(resp.text, "lxml")

一樣進行解析。

In [9]:
pd_2 = extract_articles(soup)
pd_2

,文章標題,作者,文章日期,文章連結
0,[問卦] 有人買教練課後悔的嗎,loserfatotak,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414697....
1,[問卦] 有哪一張吉卜力的圖看了就知道是誰?,dennistao,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414702....
2,[問卦] 徵召令發 會跟以色列一樣富豪幫出機票嗎,swommy,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414711....
3,Re: [爆卦] 剛剛國發會主委說我們黨團認為。。。？,MeeToo,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414746....
4,[問卦] 股市沒錢進場要怎麼調適心情,PeaceFul0804,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414795....
5,[問卦] 希特勒領導心理學規則好用嗎？,Lawleit,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414845....
6,[問卦] 打仗要準備多少食物啊,Supasizeit,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414880....
7,[問卦] 看完烏克蘭下場還跟流氓挑釁的人在想什麼?,KanzakiHAria,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414930....
8,[問卦] 大林浦遷村後 那塊地要幹麻?,kemal969,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414939....
9,[新聞] 2陸配涉武統言論限期離境 內政部長：1人仍,Yirgacheffe,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743414957....


合併兩頁的 dataframes。

In [10]:
# merge two pd
pd_articles = pd.concat([pd_1, pd_2])
pd_articles

,文章標題,作者,文章日期,文章連結
0,[新聞] 登台大港開唱喊「音樂是政治」 楊舒雅,VB6,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415548....
1,Re: [新聞] 曹興誠：若國號改「台灣共和國」是被中共,kaky,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415552....
2,[問卦] 笑哈哈LOL江老師跟Eric為什麼分道揚鑣？,shokotan,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415662....
3,[問卦] 捷運突然停車該注意什麼？,chen0625,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415770....
4,[問卦] 北捷藍線有卦嗎,MeeToo,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415894....
5,Re: [新聞] 網友問郭正亮「兩岸談判談什麼？」 黃,winnabe,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415934....
6,[新聞] 紀念日及節日修法初審通過 關鍵條文送協,ytkuang,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743415992....
7,[問卦] 黑人能吃辣嗎？,kerorobear,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743416007....
8,[問卦] 吃洋芋片會胖 不覺得完全沒道理嗎？,wang111283,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743416031....
9,[問卦] 柏原崇當年大概有多紅？,yokann,3/31,https://www.ptt.cc/bbs/Gossiping/M.1743416146....


接著，將擷取到的文章一一採集其留言。

In [11]:
results = []

for index, row in pd_articles.iterrows():
    print(f"Extracting comments of {row['文章連結']}")
    
    link = row["文章連結"]
    resp = requests.get(link, headers={"cookie": "over18=1"})
    soup = BeautifulSoup(resp.text, "lxml")

    pd_comments = extract_comments(link, soup)
    results.append(pd_comments)

pd_comments_all = pd.concat(results)
pd_comments_all

Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415548.A.6A5.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415552.A.A6E.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415662.A.4A1.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415770.A.272.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415894.A.7A5.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415934.A.C03.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743415992.A.164.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743416007.A.151.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743416031.A.F27.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743416146.A.00B.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743416206.A.5DB.html
Extracting comments of https://www.ptt.cc/bbs/Gossiping/M.1743416261.A.C5A.html
Extracting comments of https://www.ptt.c

,連結,喜好程度,留言者,留言內容,留言時間
0,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,HCYPMGO,推,11/02 21:36
1,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,LYS5566,老人才回被騙,11/02 21:36
2,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,dan5120,$$,11/02 21:37
3,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,k385476916,錢,11/02 21:37
4,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,→,QEMU,年輕人也會被騙,11/02 21:37
...,...,...,...,...,...
131,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,yylzvv,錢,03/30 20:43
132,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,jkl85621,錢,03/30 21:46
133,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,Tommy92C,有錢發的繼續連任吧,03/31 03:54
134,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,Dhack5321,辛苦了,03/31 09:25


合併成一個大的 pd_all，以 `連結` 當關聯鍵。

In [12]:
pd_all = pd.merge(pd_articles, pd_comments_all, left_on="文章連結", right_on="連結")
pd_all.drop(columns=["連結"], inplace=True)
pd_all

,文章標題,作者,文章日期,文章連結,喜好程度,留言者,留言內容,留言時間
0,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,HCYPMGO,推,11/02 21:36
1,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,LYS5566,老人才回被騙,11/02 21:36
2,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,dan5120,$$,11/02 21:37
3,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,推,k385476916,錢,11/02 21:37
4,Fw: [公告] 請留意新註冊帳號使用信件詐騙,ubcs,11/02,https://www.ptt.cc/bbs/Gossiping/M.1730554547....,→,QEMU,年輕人也會被騙,11/02 21:37
...,...,...,...,...,...,...,...,...
291,[公告] 八卦板主徵選規則修正&截止時間,ubcs,3/28,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,yylzvv,錢,03/30 20:43
292,[公告] 八卦板主徵選規則修正&截止時間,ubcs,3/28,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,jkl85621,錢,03/30 21:46
293,[公告] 八卦板主徵選規則修正&截止時間,ubcs,3/28,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,Tommy92C,有錢發的繼續連任吧,03/31 03:54
294,[公告] 八卦板主徵選規則修正&截止時間,ubcs,3/28,https://www.ptt.cc/bbs/Gossiping/M.1743146619....,推,Dhack5321,辛苦了,03/31 09:25


輸出為 `comment-detail.csv`。

In [13]:
pd_all.to_csv("comment-detail.csv", index=False)